# Main Entry Point

## Necessary Imports.

In [1]:
from typing import Optional, Tuple

In [2]:
from match import Match
from encoder import LabelEncoder
from model import ActionModel, GaitLengthPredictor, GaitValuesPredictor
from data.normlengthdata import NormLengthData
from data.normvaluesdata import NormValuesData
from data.actiondata import ActionData

from utils import save_to_json

## Const definitions.

In [3]:
SAMPLE_RATE = 0.02
SEED_SEQUENCE = [
    {
        "label": "run",
        "norm": [
            34.587473483674074,
            33.423278993730264,
            21.713335025728387,
            25.510592010144133,
            34.75362157862116,
            24.05884111539945,
            29.327713284337182,
            26.612124058201942,
            21.529695751774184,
            21.596440734113514,
        ],
    }
]

In [8]:
class Predictor:
    """Main predictor class to predict next actions."""

    def __init__(self, input_file: Optional[str] = None, match_duration_in_minutes=10):
        """Create the main predictor, it can be started using an input file or a duration of a match."""
        self.total_number_of_actions = int(
            float(match_duration_in_minutes * 60) / SAMPLE_RATE
        )
        self.encoder = LabelEncoder.load("../data/model/encoder.json")
        self.action_model = ActionModel.load(encoder=self.encoder)
        self.gait_length_model = GaitLengthPredictor.load()
        self.gait_values_model = GaitValuesPredictor.load()
        if input_file:
            self.match = Match(input_file)
            self.sequences = ActionData(self.match.clean_data()).sequences
            self.norm_lengths = NormLengthData(self.match, self.encoder)
            self.norm_values = NormValuesData(self.match, self.encoder)
        else:
            self.match = Match.from_data(SEED_SEQUENCE)
            self.sequences = [SEED_SEQUENCE[0]["label"]]
            self.norm_lengths = NormLengthData(self.match, self.encoder)
            self.norm_values = NormValuesData(self.match, self.encoder)

    def run_single_prediction(self) -> Tuple:
        """Run a single prediction from the previous predictions."""
        last_sequence = self.sequences[-1]
        last_sequence = [last_sequence] if len(last_sequence[0]) == 1 else last_sequence
        action_prediction = self.action_model.predict_standalone(last_sequence)
        encoded_prediction = self.encoder.encode(action_prediction)
        norm_length_prediction = self.gait_length_model.predict(encoded_prediction[0])
        predicted_norms = self.gait_values_model.predict(
            encoded_prediction[0], norm_length_prediction
        )
        last_sequence.append(action_prediction[0])
        self.sequences.append(last_sequence)
        self.total_number_of_actions = self.total_number_of_actions - len(
            predicted_norms
        )
        predicted_norms = [item.astype(float)[0] for item in predicted_norms]
        return {
            "label": self.encoder.decode(encoded_prediction)[0],
            "norm": predicted_norms,
        }

    def determine_ramining_time(self) -> None:
        """Print the remaining time of the whole prediction,
        for some reason logging didn't work."""
        print(
            f"Remaining Time: {(self.total_number_of_actions * SAMPLE_RATE) / 60} minutes.",
        )

    def run_prediction(self) -> None:
        """Run a prediction task and store the results to `result.json`."""
        result = [SEED_SEQUENCE[0]]
        while self.total_number_of_actions > 0:
            self.determine_ramining_time()
            single_prediction = self.run_single_prediction()
            result.append(single_prediction)

        save_to_json(result, "result.json")

In [9]:
predictor = Predictor()

In [10]:
predictor.run_single_prediction()

1/1 [==============================] - 1s 1s/step


{'label': 'walk',
 'norm': [19.0,
  19.0,
  23.0,
  23.0,
  19.0,
  22.0,
  27.0,
  22.0,
  18.0,
  20.0,
  21.0,
  25.0,
  23.0,
  24.0,
  20.0,
  27.0,
  33.0,
  25.0,
  24.0,
  29.0,
  22.0,
  37.0,
  21.0,
  32.0,
  19.0,
  23.0,
  29.0,
  19.0,
  25.0,
  26.0]}

In [11]:
predictor.run_prediction()

Remaining Time: 9.99 minutes.
1/1 [==============================] - 0s 31ms/step
Remaining Time: 9.971666666666668 minutes.
1/1 [==============================] - 0s 23ms/step
Remaining Time: 9.939666666666666 minutes.
1/1 [==============================] - 0s 31ms/step
Remaining Time: 9.921333333333333 minutes.
1/1 [==============================] - 0s 23ms/step
Remaining Time: 9.906 minutes.
1/1 [==============================] - 0s 29ms/step
Remaining Time: 9.863 minutes.
1/1 [==============================] - 0s 31ms/step
Remaining Time: 9.844666666666667 minutes.
1/1 [==============================] - 0s 24ms/step
Remaining Time: 9.826333333333334 minutes.
1/1 [==============================] - 0s 23ms/step
Remaining Time: 9.807666666666668 minutes.
1/1 [==============================] - 0s 22ms/step
Remaining Time: 9.800333333333333 minutes.
1/1 [==============================] - 0s 28ms/step
Remaining Time: 9.782 minutes.
1/1 [==============================] - 0s 23ms/step
Rema